# read polymer data

In [1]:
import polymer
data =  polymer.read("polymer.csv")

# split to train-test

In [2]:
p = 0.8   # percentage for training
splits, s = [], 10  # get s splits

for i in range(s):
    train, test = data.split(p, i)
    splits.append((train, test))

# [feedforward neural network](https://en.wikipedia.org/wiki/Feedforward_neural_network)

In [3]:
import torch, torch.nn as nn, torch.nn.functional as F

class FeedNet(nn.Module):
    
    def __init__(self, h):
        super(FeedNet, self).__init__()
        self.layer1 = nn.Linear(3, h)
        self.layer2 = nn.Linear(h, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        out = F.relu(self.layer1(x))
        out = self.layer2(out)
        out = self.sigmoid(out)
        return out

# initialise regression model

In [4]:
from skorch import NeuralNetRegressor
e = 20 # number of epochs
net = NeuralNetRegressor(FeedNet, max_epochs=e, verbose = 0)

# import useful modules

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm
import numpy as np

# select model, regress

In [6]:
mse, r2 = [], []
for i in tqdm(range(s)):
    train, test = splits[i]
    X = np.array(train["X"]).astype(np.float32)
    Y = np.array(train["Y"]).reshape(-1, 1).astype(np.float32)
    
    '''
    select the best model
    '''
    params = {'module__h': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]}
    GS = GridSearchCV(net, params, refit=False, cv=3, scoring='r2', verbose=0)
    _ = GS.fit(X, Y)
    
    '''
    train the best model
    '''
    FFNN = FeedNet(GS.best_params_['module__h'])
    best = NeuralNetRegressor(FFNN, max_epochs=e, verbose = 0)
    _ = best.fit(X, Y)
    
    '''
    test the best model
    '''
    X = np.array(test["X"]).astype(np.float32)
    Y = np.array(test["Y"]).reshape(-1, 1).astype(np.float32)
    Z = best.predict(X)
    
    mse.append(mean_squared_error(Y, Z))
    r2.append(r2_score(Y.squeeze(), Z.squeeze()))

print("mse: ", round(np.mean(mse), 3), "+-", round(np.std(mse), 3))
print("r2: ", round(np.mean(r2), 3), "+-", round(np.std(r2), 3))

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:25<00:00,  2.50s/it]


mse:  0.192 +- 0.24
r2:  -2.749 +- 3.524


# [run all cells](https://stackoverflow.com/questions/33143753/jupyter-ipython-notebooks-shortcut-for-run-all)

In [7]:
%%javascript

Jupyter.keyboard_manager.command_shortcuts.add_shortcut('r', {
    help : 'run all cells',
    help_index : 'zz',
    handler : function (event) {
        IPython.notebook.execute_all_cells();
        return false;
    }}
);

<IPython.core.display.Javascript object>